<a href="https://colab.research.google.com/github/ozturkergin/ozturkergin/blob/main/TEFAS_PowerBI_Import_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip3 install -r requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.6/446.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 7.3 MB/s eta 0:00:00


In [5]:
import requests
import pandas as pd
import math
import concurrent.futures
import time

from datetime import datetime, timedelta, date
from typing import Dict, List, Optional, Union
from marshmallow import Schema, fields, EXCLUDE, pre_load, post_load
from bs4 import BeautifulSoup
from fp.fp import FreeProxy

# Special thanks to https://github.com/burakyilmaz321

class InfoSchema(Schema):
    code = fields.String(data_key="FONKODU", allow_none=True)
    fonunvantip = fields.String(data_key="FONUNVANTIP", allow_none=True)
    date = fields.Date(data_key="TARIH", allow_none=True)
    price = fields.Float(data_key="FIYAT", allow_none=True)
    title = fields.String(data_key="FONUNVAN", allow_none=True)
    market_cap = fields.Float(data_key="PORTFOYBUYUKLUK", allow_none=True)
    number_of_shares = fields.Float(data_key="TEDPAYSAYISI", allow_none=True)
    number_of_investors = fields.Float(data_key="KISISAYISI", allow_none=True)

    @pre_load
    def pre_load_hook(self, input_data, **kwargs):
        seconds_timestamp = int(input_data["TARIH"]) / 1000
        input_data["TARIH"] = date.fromtimestamp(seconds_timestamp).isoformat()
        return input_data

    @post_load
    def post_load_hool(self, output_data, **kwargs):
        output_data = {f: output_data.setdefault(f) for f in self.fields}
        return output_data

    class Meta:
        unknown = EXCLUDE

class tefas_get:
    root_url = "https://www.tefas.gov.tr"
    info_endpoint = "/api/DB/BindHistoryInfo"
    concurrently = False
    use_Proxy = False
    fon_type = "YAT"
    proxies = None

    @staticmethod
    def get_combobox_items(url, select_id):
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception(f"Failed to fetch the URL: {response.status_code}")

        soup = BeautifulSoup(response.content, 'html.parser')
        select_element = soup.find('select', id=select_id)

        if not select_element:
            raise Exception(f"Select element with id '{select_id}' not found")

        options = select_element.find_all('option')
        options = list(filter(None, options))

        items = []
        for option in options:
            value = option.get('value')
            items.append(value)

        items.remove('')

        return items

    def fetch_info(self, fonunvantip, start_date_initial, end_date_initial):
        counter = 1
        start_date = start_date_initial
        end_date = end_date_initial
        range_date = end_date_initial - start_date_initial
        range_interval = 90
        info_schema = InfoSchema(many=True)
        info_result = pd.DataFrame()

        if range_date.days > range_interval :
            counter = range_date.days / range_interval
            counter = math.ceil(counter)
            end_date = start_date + timedelta(days=range_interval)

        while counter > 0:
            counter -= 1

            data = {
                    "fontip": self.fon_type,
                    "bastarih": self._parse_date(start_date),
                    "bittarih": self._parse_date(end_date),
                    "fonunvantip": fonunvantip,
                    "fonkod": "",
                  }

            info = self._do_post(data)
            info = info_schema.load(info)
            info = pd.DataFrame(info, columns=info_schema.fields.keys())
            #info['fonunvantip'] = fonunvantip.replace("(","").replace(")","").replace(" ","")
            info['fonunvantip'] = fonunvantip
            #info = info[info['price'] != 0]
            if not info.empty :
                info_result = pd.concat([info_result, info])
                info_result = info_result.reset_index(drop=True)
                info = info.reset_index(drop=True)

            if counter > 0 :
                start_date = end_date + timedelta(days=1)
                end_date = end_date + timedelta(days=range_interval)
                if end_date > end_date_initial :
                    end_date = end_date_initial

        return info_result

    def fetch_info_serial(self, fonunvantips, start_date_initial, end_date_initial):
        merged = pd.DataFrame()

        for fonunvantip in fonunvantips:
            info = self.fetch_info(fonunvantip, start_date_initial, end_date_initial)
            if not info.empty :
                merged = pd.concat([merged, info])
                print(f"{fonunvantip} - {len(info)} records added total records: {len(merged)} " )

        return merged

    def fetch_info_concurrently(self, fonunvantips, start_date_initial, end_date_initial):
        with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
            merged = pd.DataFrame()
            # Submit all tasks to the executor
            self.concurrently = True
            futures = {executor.submit(self.fetch_info, fonunvantip, start_date_initial, end_date_initial): fonunvantip for fonunvantip in fonunvantips}

            # Retrieve results as they complete
            for future in concurrent.futures.as_completed(futures):
                info = future.result()
                merged = pd.concat([merged, info])
                print(f"{future} - {len(info)} records added total records: {len(merged)} " )

            return merged

    def fetch(
        self,
        start: Union[str, datetime],
        end: Optional[Union[str, datetime]] = None,
        columns: Optional[List[str]] = None,
        unvantip: bool = False,
    ) -> pd.DataFrame:

        start_date_initial = datetime.strptime(start, "%Y-%m-%d")
        end_date_initial = datetime.strptime(end or start, "%Y-%m-%d")

        merged = pd.DataFrame()
        fonunvantips = [""]
        if unvantip :
            fonunvantips = self.get_combobox_items(url="https://www.tefas.gov.tr/TarihselVeriler.aspx", select_id="DropDownListFundTypeExplanationYAT")

        if self.use_Proxy :
            proxy_address = self.get_free_proxy()
            self.proxies = {"http": proxy_address, "https": proxy_address}
            print(self.proxies)
        else :
            self.proxies = None

        if self.concurrently :
            merged = self.fetch_info_concurrently(fonunvantips, start_date_initial, end_date_initial)
        else :
            merged = self.fetch_info_serial(fonunvantips, start_date_initial, end_date_initial)

        merged = merged[columns] if columns and not merged.empty else merged

        return merged

    def get_free_proxy(self):
        proxy_address = FreeProxy(timeout=1, rand=True, https=True).get()
        return proxy_address

    def _do_post(self, data: Dict[str, str]) -> Dict[str, str]:
        timestamp = int(time.time() * 1000)  # Get current timestamp in milliseconds
        headers = {
         "Connection": "keep-alive",
         "Cache-Control": "no-cache",
         "Pragma": "no-cache",
         "X-Requested-With": "XMLHttpRequest",
         "Sec-Fetch-Mode": "cors",
         "Sec-Fetch-Site": "same-origin",
         "Accept-Language": "tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7",
         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
         "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
         "Accept": "application/json, text/javascript, */*; q=0.01",
         "Origin": "https://www.tefas.gov.tr",
         "Referer": f"https://www.tefas.gov.tr/TarihselVeriler.aspx?timestamp={timestamp}" ,
         }

        response = requests.post(
             url=f"{self.root_url}/{self.info_endpoint}",
             data=data,
             proxies=self.proxies,
             headers=headers,
         )
        # Check the response status code and content
        if response.status_code != 200:
            print(f"Request failed with status code: {response.status_code}")
            print(f"Response content: {response.text}")
            return {}  # Return an empty dictionary if the request failed
        try:
            return response.json().get("data", {})
        except ValueError as e:
            print(f"Error decoding JSON response: {e}")
            print(f"Response content: {response.text}")
            return {}

    def _parse_date(self, date: Union[str, datetime]) -> str:
        if isinstance(date, datetime):
            formatted = datetime.strftime(date, "%d.%m.%Y")
        elif isinstance(date, str):
            try:
                parsed = datetime.strptime(date, "%Y-%m-%d")
            except ValueError as exc:
                raise ValueError(
                    "Date string format is incorrect. " "It should be `YYYY-MM-DD`"
                ) from exc
            else:
                formatted = datetime.strftime(parsed, "%d.%m.%Y")
        else:
            raise ValueError(
                "`date` should be a string like 'YYYY-MM-DD' "
                "or a `datetime.datetime` object."
            )
        return formatted

tefas = tefas_get()

In [6]:
time_delta = 450
start_date_calc = date.today() - timedelta(days=time_delta)
date_start = start_date_calc.strftime("%Y-%m-%d")
date_end = date.today().strftime("%Y-%m-%d")

fetched_data = pd.DataFrame()
fetched_data = tefas.fetch(start=date_start, end=date_end, columns=["code", "date", "price", "market_cap", "number_of_shares", "number_of_investors"], unvantip=False)
fetched_data['date'] = pd.to_datetime(fetched_data['date'], errors='coerce')
fetched_data['date'].dt.strftime('%Y-%m-%d')
fetched_data['date'] = fetched_data['date'].dt.date
fetched_data['price'].astype(float,False)
fetched_data.rename(columns={'price': 'close'}, inplace=True)
fetched_data.rename(columns={'code': 'symbol'}, inplace=True)
fetched_data['market_cap'].astype(float,False)
fetched_data['number_of_shares'].astype(float,False)
fetched_data['number_of_investors'].astype(float,False)
fetched_data['market_cap_per_investors'] = fetched_data['market_cap'] / fetched_data['number_of_investors']
fetched_data[(fetched_data!=0)&(pd.isnull(fetched_data))]
fetched_data = fetched_data.sort_values(['symbol', 'date'])
fetched_data['open'] = fetched_data.groupby('symbol')['close'].shift(1)
fetched_data['high'] = fetched_data[['open', 'close']].max(axis=1)
fetched_data['low'] = fetched_data[['open', 'close']].min(axis=1)
fetched_data = fetched_data.dropna()
fetched_data.to_csv('tefas.csv', encoding='utf-8-sig', index=False)
fetched_data

 - 440556 records added total records: 440556 


,symbol,date,close,market_cap,number_of_shares,number_of_investors,market_cap_per_investors,open,high,low
76707,AAK,2023-06-13,11.463897,95064137.76,8292480.0,526.0,1.807303e+05,11.457887,11.463897,11.457887
75422,AAK,2023-06-14,11.364256,89237749.29,7852494.0,528.0,1.690109e+05,11.463897,11.463897,11.364256
74137,AAK,2023-06-15,11.308113,88743098.15,7847737.0,528.0,1.680740e+05,11.364256,11.364256,11.308113
72851,AAK,2023-06-16,11.388840,82648500.82,7256973.0,527.0,1.568283e+05,11.308113,11.388840,11.308113
71563,AAK,2023-06-19,11.392882,77670608.92,6817468.0,526.0,1.476628e+05,11.388840,11.392882,11.388840
...,...,...,...,...,...,...,...,...,...,...
354049,ZZL,2024-08-26,47.636622,24262236.45,509319.0,15.0,1.617482e+06,47.561080,47.636622,47.561080
352406,ZZL,2024-08-27,47.546479,24216325.06,509319.0,15.0,1.614422e+06,47.636622,47.636622,47.546479
350763,ZZL,2024-08-28,47.505592,24195500.59,509319.0,15.0,1.613033e+06,47.546479,47.546479,47.505592
349120,ZZL,2024-08-29,47.593184,24240112.67,509319.0,15.0,1.616008e+06,47.505592,47.593184,47.505592


In [35]:
import pandas_ta as ta
import pandas as pd
import numpy as np

fetched_data=pd.read_csv('tefas.csv')
fetched_data['date'] = pd.to_datetime(fetched_data['date'])
fetched_data['qty'] = 100/fetched_data['close']
fetched_data['year'] = fetched_data['date'].dt.year
fetched_data['week_no'] = fetched_data['date'].dt.isocalendar().week.astype(str).str.zfill(2)
fetched_data['year_week'] = fetched_data['year'].astype(str) +'-'+ fetched_data['week_no'].astype(str)
fetched_data['day_of_week'] = fetched_data['date'].apply(lambda x: x.strftime('%A'))
#idx = fetched_data.groupby(['symbol', 'year_week'])['date'].idxmax()
#max_prices = fetched_data.loc[idx, ['symbol', 'year_week', 'close']]
#max_prices = max_prices.rename(columns={'close': 'price_at_week_close'})
#fetched_data = fetched_data.merge(max_prices, on=['symbol', 'year_week'], how='left')
#fetched_data['valuation_at_week_close'] = fetched_data['price_at_week_close'] * fetched_data['qty']

def calculate_ta(group):
    SMA_FAST = 50
    SMA_SLOW = 200
    RSI_PERIOD = 14
    RSI_AVG_PERIOD = 15

    group['ema12'] = ta.ema(group['close'], length=12, fillna=group.close)
    group['ema26'] = ta.ema(group['close'], length=26, fillna=group.close)
    group['sma_f'] = group['close'].rolling(SMA_FAST).mean()
    group['sma_s'] = group['close'].rolling(SMA_SLOW).mean()
    group['rsi'] = ta.rsi(group.close, RSI_PERIOD)
    group['sma_r'] = group.rsi.rolling(RSI_AVG_PERIOD).mean()
    group['sma'] = np.where(group.sma_f > group.sma_s, 1, 0)
    group['rsi_test'] = np.where((group.rsi < 50) & (group.rsi > group.rsi.shift(1)), 1, 0)
    return group

fetched_data = fetched_data.reset_index(drop=True)
fetched_data.sort_values(by=['symbol', 'date'], inplace=True)
fetched_data = fetched_data.groupby(['symbol']).apply(calculate_ta)
fetched_data = fetched_data.fillna(0)
fetched_data.to_csv('tefas_transformed.csv', encoding='utf-8-sig', index=False)
fetched_data

<ipython-input-35-9cb9673b484d>:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fetched_data = fetched_data.groupby(['symbol']).apply(calculate_ta)


symbol       date      close   market_cap  number_of_shares  \
symbol                                                                      
AAK    0         AAK 2023-06-13  11.463897  95064137.76         8292480.0   
       1         AAK 2023-06-14  11.364256  89237749.29         7852494.0   
       2         AAK 2023-06-15  11.308113  88743098.15         7847737.0   
       3         AAK 2023-06-16  11.388840  82648500.82         7256973.0   
       4         AAK 2023-06-19  11.392882  77670608.92         6817468.0   
...              ...        ...        ...          ...               ...   
ZZL    422844    ZZL 2024-08-26  47.636622  24262236.45          509319.0   
       422845    ZZL 2024-08-27  47.546479  24216325.06          509319.0   
       422846    ZZL 2024-08-28  47.505592  24195500.59          509319.0   
       422847    ZZL 2024-08-29  47.593184  24240112.67          509319.0   
       422848    ZZL 2024-09-02  47.665448  24276918.47          509319.0   

               number_of_investors  market_cap_per_investors       open  \
symbol                                                                    
AAK    0                     526.0              1.807303e+05  11.457887   
       1                     528.0              1.690109e+05  11.463897   
       2                     528.0              1.680740e+05  11.364256   
       3                     527.0              1.568283e+05  11.308113   
       4                     526.0              1.476628e+05  11.388840   
...                            ...                       ...        ...   
ZZL    422844                 15.0              1.617482e+06  47.561080   
       422845                 15.0              1.614422e+06  47.636622   
       422846                 15.0              1.613033e+06  47.546479   
       422847                 15.0              1.616008e+06  47.505592   
       422848                 15.0              1.618461e+06  47.593184   

                    high        low  ...  year_week  day_of_week      ema12  \
symbol                               ...                                      
AAK    0       11.463897  11.457887  ...    2023-24      Tuesday  11.463897   
       1       11.463897  11.364256  ...    2023-24    Wednesday  11.364256   
       2       11.364256  11.308113  ...    2023-24     Thursday  11.308113   
       3       11.388840  11.308113  ...    2023-24       Friday  11.388840   
       4       11.392882  11.388840  ...    2023-25       Monday  11.392882   
...                  ...        ...  ...        ...          ...        ...   
ZZL    422844  47.636622  47.561080  ...    2024-35       Monday  47.063027   
       422845  47.636622  47.546479  ...    2024-35      Tuesday  47.137405   
       422846  47.546479  47.505592  ...    2024-35    Wednesday  47.194049   
       422847  47.593184  47.505592  ...    2024-35     Thursday  47.255454   
       422848  47.665448  47.593184  ...    2024-36       Monday  47.318530   

                   ema26      sma_f      sma_s        rsi      sma_r  sma  \
symbol                                                                      
AAK    0       11.463897   0.000000   0.000000   0.000000   0.000000    0   
       1       11.364256   0.000000   0.000000   0.000000   0.000000    0   
       2       11.308113   0.000000   0.000000   0.000000   0.000000    0   
       3       11.388840   0.000000   0.000000   0.000000   0.000000    0   
       4       11.392882   0.000000   0.000000   0.000000   0.000000    0   
...                  ...        ...        ...        ...        ...  ...   
ZZL    422844  46.453210  45.425124  39.886106  87.222983  80.501087    1   
       422845  46.534193  45.512519  39.960311  82.689124  80.259630    1   
       422846  46.606149  45.598550  40.033273  80.641577  80.809489    1   
       422847  46.679262  45.686090  40.106152  81.687722  81.304622    1   
       422848  46.752313  45.766952  40.179354  82.526678  81.781334    1   

      

In [5]:
time_delta = 15
start_date_calc = date.today() - timedelta(days=time_delta)
date_start = start_date_calc.strftime("%Y-%m-%d")
date_end = date.today().strftime("%Y-%m-%d")
fetched_data_agg = tefas.fetch(start=date_start, end=date_end, columns=["code", "date", "price", "fonunvantip", "title"], unvantip=True)
fetched_data_agg.drop_duplicates(subset=['code', 'fonunvantip'], ignore_index=True, inplace=True)
fon_table = fetched_data_agg.pivot_table(index=['title', 'code'], columns='fonunvantip', aggfunc='size', fill_value=0)
fon_table.reset_index(inplace=True)
fon_table = fon_table.replace(0, False)
fon_table = fon_table.replace(1, True)
fon_table.rename(columns={'code': 'symbol'}, inplace=True)
fon_table['symbol_with_title'] = fon_table['symbol'].astype(str) +' - '+ fon_table['title'].astype(str)
fon_table.to_csv('fon_table.csv', encoding='utf-8-sig', index=False)
fon_table

Agresif Değişken - 40 records added total records: 40 
Alternatif - 30 records added total records: 70 
Altın - 300 records added total records: 370 
Atak Dinamik Büyüme Değişken - 10 records added total records: 380 
Borçlanma Araçları - 750 records added total records: 1130 
Çalışanlarına Yönelik - 20 records added total records: 1150 
Çoklu Varlık - 110 records added total records: 1260 
Değişken - 1340 records added total records: 2600 
Dengeli Değişken - 30 records added total records: 2630 
Diğer Fon Sepeti - 20 records added total records: 2650 
Döviz - 5060 records added total records: 7710 
Döviz Cinsinden İhraç (Dolar) - 4540 records added total records: 12250 
Döviz Cinsinden İhraç (Euro) - 500 records added total records: 12750 
Emtia - 80 records added total records: 12830 
Endeks - 160 records added total records: 12990 
Endeks Hisse Senedi - 40 records added total records: 13030 
Eurobond - 90 records added total records: 13120 
Fon Sepeti - 787 records added total recor

fonunvantip,title,symbol,Agresif Değişken,Alternatif,Altın,Atak Dinamik Büyüme Değişken,Borçlanma Araçları,Dengeli Değişken,Değişken,Diğer Fon Sepeti,...,Serbest,Sürdürülebilirlik Fonları,Uzun Vadeli,Yabancı,Yabancı Fon Sepeti,Çalışanlarına Yönelik,Çoklu Varlık,Özel,İştirak,symbol_with_title
0,AK PORTFÖY 0-5 YIL VADELİ SERBEST (DÖVİZ) FON,AJK,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,AJK - AK PORTFÖY 0-5 YIL VADELİ SERBEST (DÖVİZ...
1,AK PORTFÖY 2025 SERBEST (DÖVİZ) FON,YRB,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,YRB - AK PORTFÖY 2025 SERBEST (DÖVİZ) FON
2,AK PORTFÖY 2028 SERBEST (DÖVİZ) FON,SER,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,SER - AK PORTFÖY 2028 SERBEST (DÖVİZ) FON
3,AK PORTFÖY ADOC SERBEST (DÖVİZ) ÖZEL FON,CJF,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,True,False,CJF - AK PORTFÖY ADOC SERBEST (DÖVİZ) ÖZEL FON
4,AK PORTFÖY AK SANDIK SERBEST ÖZEL FON,SSO,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,True,False,SSO - AK PORTFÖY AK SANDIK SERBEST ÖZEL FON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,İŞ PORTFÖY İHRACATÇI ŞİRKETLER HİSSE SENEDİ (T...,IHT,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,IHT - İŞ PORTFÖY İHRACATÇI ŞİRKETLER HİSSE SEN...
1639,İŞ PORTFÖY İKİNCİ PARA PİYASASI (TL) FONU,IOO,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,IOO - İŞ PORTFÖY İKİNCİ PARA PİYASASI (TL) FONU
1640,İŞ PORTFÖY İKİNCİ SERBEST (TL) FON,ILI,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,ILI - İŞ PORTFÖY İKİNCİ SERBEST (TL) FON
1641,İŞ PORTFÖY İŞ BANKASI İŞTİRAKLERİ ENDEKSİ HİSS...,TI3,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,TI3 - İŞ PORTFÖY İŞ BANKASI İŞTİRAKLERİ ENDEKS...
